In [10]:
pip install transformers datasets torch scikit-learn pandas


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


# Download dataset

In [11]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment-analysis-dataset


In [12]:
import pandas as pd

csv_path = path+"/train.csv"
data = pd.read_csv(csv_path, encoding='latin1')
print("\nSố dòng trong mỗi cột:")
print(len(data[['text', 'sentiment']]))

print("\nSố dòng thiếu trong mỗi cột:")
print(data[['text', 'sentiment']].isna().sum())

data_cleaned = data.dropna(subset=['text', 'sentiment'])

data_cleaned = data_cleaned[data_cleaned['text'].str.strip() != ""]
data_cleaned = data_cleaned[data_cleaned['sentiment'].str.strip() != ""]

data_cleaned.to_csv("cleaned_train.csv", index=False)

#================================================

csv_path = path+"/test.csv"
data = pd.read_csv(csv_path, encoding='latin1')

print("\nSố dòng thiếu trong mỗi cột:")
print(data[['text', 'sentiment']].isna().sum())

data_cleaned = data.dropna(subset=['text', 'sentiment'])

data_cleaned = data_cleaned[data_cleaned['text'].str.strip() != ""]
data_cleaned = data_cleaned[data_cleaned['sentiment'].str.strip() != ""]

data_cleaned.to_csv("cleaned_test.csv", index=False)


Số dòng trong mỗi cột:
27481

Số dòng thiếu trong mỗi cột:
text         1
sentiment    0
dtype: int64

Số dòng thiếu trong mỗi cột:
text         1281
sentiment    1281
dtype: int64


In [13]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv('/kaggle/working/cleaned_train.csv', encoding='latin-1')  
tdf = pd.read_csv('/kaggle/working/cleaned_test.csv', encoding='latin-1')

print(df.head())

print(df.columns)

df = df[['text', 'sentiment']]
tdf = tdf[['text', 'sentiment']]

def convert_sentiment_label(x):
    if x == 'positive':
        return 1
    elif x == 'negative':
        return 0
    else:
        return 2

df['sentiment'] = df['sentiment'].apply(convert_sentiment_label)
tdf['sentiment'] = tdf['sentiment'].apply(convert_sentiment_label)

df['text'] = df['text'].astype(str)
tdf['text'] = tdf['text'].astype(str)

train_texts = df['text']
train_labels = df['sentiment']
val_texts= tdf['text']
val_labels = tdf['sentiment']

print("Train size:", len(train_texts))
print("Test size:", len(val_texts))

       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment Time of Tweet Age of User  \
0  I`d have responded, if I were going   neutral       morning        0-20   
1                             Sooo SAD  negative          noon       21-30   
2                          bullying me  negative         night       31-45   
3                       leave me alone  negative       morning       46-60   
4                        Sons of ****,  negative          noon       60-70   

       Country  Population -2020  Land Area (KmÂ²)  Density (P/KmÂ²)  
0  Afghanistan          38928346          652860.0 

In [19]:
from transformers import BertTokenizer, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
print("Tokenizer loaded successfully!")

def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=220, return_tensors="pt")

train_encodings = tokenize_function(list(train_texts))
val_encodings = tokenize_function(list(val_texts))


Tokenizer loaded successfully!


In [20]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
print('run complete')


run complete


In [16]:
!pip install --upgrade peft

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
from peft import PrefixEncoder, PrefixTuningConfig, get_peft_model
from transformers import BertForSequenceClassification
import torch
from torch import nn
import os

config = PrefixTuningConfig(
    peft_type="PREFIX_TUNING",
    task_type="SEQ_CLS",
    num_virtual_tokens=100,
    token_dim=1024,
    num_transformer_submodules=1,
    num_attention_heads=16,
    encoder_hidden_size=1024 
)

def defineNewModel(config = config):
    base_model = BertForSequenceClassification.from_pretrained(
            "bert-large-uncased", num_labels=3)

    prefix_model = get_peft_model(base_model, config)
    prefix_model.print_trainable_parameters()
    return prefix_model

In [ ]:
from torch.nn import CrossEntropyLoss
from torch import cuda
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import time
import random
from torch.utils.data import Subset
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification
from collections import Counter

device = 'cuda' if cuda.is_available() else 'cpu'
data_len_list = [100, 200, 300, 400, 500, 1000, 10000]
epochs = 10
batch_size = 32
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

def EvalModel(model):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Validation Trained Model Accuracy: {accuracy:.4f}")
    print("===========================================")

# Training for many data_length
for data_length in data_len_list:
    model = defineNewModel()

    for name,param in model.named_parameters():
        if param.requires_grad==True:
            print("Traing Param: ", name)

    model.to(device)
    model.train()
    print("Training Data Length: ", data_length)
    # Create random indices
    random_indices = random.sample(range(len(train_dataset)), data_length)
    train_subset = Subset(train_dataset, random_indices)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=5e-4)
    
    label_counts = Counter()
    for batch in train_loader:
        labels = batch['labels']
        label_counts.update(labels.tolist())
    for label, count in label_counts.items():
        print(f"Label {label}: {count} samples")

    for epoch in range(epochs):
        start_time = time.time()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        for batch in train_loader:
            optimizer.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch["labels"]
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += labels.size(0)
        epoch_time = time.time() - start_time
        avg_loss = total_loss / len(train_loader)
        accuracy = correct_predictions / total_samples * 100
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}, "
              f"Accuracy: {correct_predictions / total_samples:.4f}, "
              f"Time: {epoch_time:.2f} seconds")

    # Eval model =====================================================
    EvalModel(model)


    



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 4,918,275 || all params: 340,063,238 || trainable%: 1.4463
Traing Param:  base_model.classifier.modules_to_save.default.weight
Traing Param:  base_model.classifier.modules_to_save.default.bias
Traing Param:  prompt_encoder.default.embedding.weight
Training Data Length:  100


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Label 2: 48 samples
Label 0: 34 samples
Label 1: 18 samples
Epoch 1/10, Loss: 1.066273808479309, Accuracy: 0.5300, Time: 5.49 seconds
Epoch 2/10, Loss: 1.164434552192688, Accuracy: 0.4500, Time: 4.86 seconds
Epoch 3/10, Loss: 1.1195420026779175, Accuracy: 0.4500, Time: 4.87 seconds
Epoch 4/10, Loss: 0.9903809875249863, Accuracy: 0.4800, Time: 4.87 seconds
Epoch 5/10, Loss: 1.0555797219276428, Accuracy: 0.4600, Time: 4.86 seconds
Epoch 6/10, Loss: 0.9894406348466873, Accuracy: 0.4800, Time: 4.87 seconds
Epoch 7/10, Loss: 0.9628000259399414, Accuracy: 0.4400, Time: 4.86 seconds
Epoch 8/10, Loss: 1.066466599702835, Accuracy: 0.4600, Time: 4.86 seconds
Epoch 9/10, Loss: 1.0548924803733826, Accuracy: 0.4700, Time: 4.87 seconds
Epoch 10/10, Loss: 1.063894808292389, Accuracy: 0.4900, Time: 4.87 seconds
Validation Trained Model Accuracy: 0.3698


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1/10, Loss: 1.1157566085457802, Accuracy: 0.3650, Time: 48.69 seconds
Epoch 2/10, Loss: 1.1201768442988396, Accuracy: 0.3520, Time: 48.73 seconds
Epoch 3/10, Loss: 1.0888073593378067, Accuracy: 0.3910, Time: 48.76 seconds
Epoch 4/10, Loss: 1.0820408929139376, Accuracy: 0.4000, Time: 48.75 seconds
Epoch 5/10, Loss: 1.040732093155384, Accuracy: 0.4410, Time: 48.74 seconds
Epoch 6/10, Loss: 0.9986618589609861, Accuracy: 0.4840, Time: 48.75 seconds
Epoch 7/10, Loss: 0.9545777617022395, Accuracy: 0.5150, Time: 48.76 seconds
Epoch 8/10, Loss: 0.908342182636261, Accuracy: 0.5680, Time: 48.75 seconds
Epoch 9/10, Loss: 0.8552126847207546, Accuracy: 0.5910, Time: 48.76 seconds
Epoch 10/10, Loss: 0.8398021534085274, Accuracy: 0.5990, Time: 48.77 seconds
Validation Trained Model Accuracy: 0.6242


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 4,918,275 || all params: 340,063,238 || trainable%: 1.4463
Traing Param:  base_model.classifier.modules_to_save.default.weight
Traing Param:  base_model.classifier.modules_to_save.default.bias
Traing Param:  prompt_encoder.default.embedding.weight
Training Data Length:  10000


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Label 0: 2877 samples
Label 2: 4080 samples
Label 1: 3043 samples


In [ ]:

#======================
model = BertForSequenceClassification.from_pretrained(
            "bert-large-uncased", num_labels=3).to("cuda")
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Validation Base Model Accuracy: {accuracy:.4f}")